<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/%E9%89%85%E4%BA%A8%E7%B6%B2%E6%96%87%E5%AD%97%E6%8E%A2%E5%8B%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%capture
!pip install transformers
!pip install gdown
!pip install opencc-python-reimplemented
!pip install googletrans==3.1.0a0

In [27]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=65568b86a14aae0dfa9652e9317b34dc61c9e5f22f1ded7c46bcf53dd96a4726
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [2]:
import torch
import numpy as np
import pandas as pd

In [3]:
from opencc import OpenCC
cc = OpenCC('t2s')

In [4]:
#!gdown https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
#!unzip /content/FinBERT_L-12_H-768_A-12_pytorch.zip

Downloading...
From: https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
To: /content/FinBERT_L-12_H-768_A-12_pytorch.zip
100% 382M/382M [00:02<00:00, 135MB/s]
Archive:  /content/FinBERT_L-12_H-768_A-12_pytorch.zip
   creating: FinBERT_L-12_H-768_A-12_pytorch/
  inflating: FinBERT_L-12_H-768_A-12_pytorch/bert_config.json  
  inflating: FinBERT_L-12_H-768_A-12_pytorch/vocab.txt  
  inflating: FinBERT_L-12_H-768_A-12_pytorch/pytorch_model.bin  


# 財金BERT

In [ ]:
from transformers import BertModel, BertTokenizer
PATH = '/content/FinBERT_L-12_H-768_A-12_pytorch'

tokenizer = BertTokenizer.from_pretrained(PATH ,local_files_only=True)
model = BertModel.from_pretrained(PATH ,local_files_only=True)

In [ ]:
sector_dict = {
'健護': '醫療保健服務類股',
'科技': '科技類股', 
'公用事業': '公用事業類股',
'生技':'生技類股', 
'能源':'能源類股',
'替代能源':'替代能源類股',
'消費':'消費品類股',
'金融':'金融類股', 
'礦業':'礦業',
'天然資源':'天然資源', 
'貴金屬現貨':'貴金屬',
'農產品':'農產品',
'貴金屬類股':'貴金屬',
}

In [ ]:
mkt_dict= [
'新興市場債',
'高收益債',
'全球債',
'投資等級公司債',
'美國公債',
'亞債',
'REITs不動產投資信託',
'新臺幣類貨幣',
'外幣類貨幣',
'亞太股市', '美國股市', '全球新興市場', 
'全球型股市', '大中華股市', '中國A股', '印度股市','歐洲股市', 
'東南亞股市', '拉丁美洲股市', 
'韓國股市', '新興歐洲股市', '德國股市', '台灣股市','俄羅斯股市',
'中國H股', '日本股市', '英國股市','泰國股市',
'香港股市', '澳洲股市', '印尼股市', '巴西股市', '歐非中東股市'
]


In [ ]:
mkt_dict = {x:x for x in mkt_dict}
mkt_dict['亞債'] = '亞洲債券'
mkt_dict['REITs不動產投資信託'] = '不動產'

In [ ]:
total_sec_dict = {**sector_dict, **mkt_dict}

In [ ]:
sentences = list(set(total_sec_dict.values()))
sentences = [cc.convert(x) for x in sentences]

In [ ]:
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
x = outputs['last_hidden_state']
np.savetxt('bert_mkt_emb.tsv', x[:,0].cpu().detach().numpy(), delimiter="\t")
pd.DataFrame(sentences).to_csv('bert_mkt_emb_meta.tsv', header=False, index=False, sep="\t")

# 鉅亨網爬蟲


In [13]:
import requests
from bs4 import BeautifulSoup


In [14]:
res = requests.get('https://news.cnyes.com/news/cat/wd_stock')

In [15]:
#res2 = requests.get('https://blog.anuefund.com/page/2/?utm_campaign=bloglink&utm_medium=fund_mainlist&utm_source=anuefundapp')

In [16]:
soup = BeautifulSoup(res.text, 'html.parser')

In [19]:
news_list = soup.find_all("a", attrs={"class":"_1Zdp"})

In [ ]:
[x['href'] for x in news_list]
[x.find('h3').get_text(strip=True) for x in news_list]

## .get_text(strip=True) 可避免不必用空白符亂碼

In [74]:
from fake_useragent import UserAgent

ua = UserAgent()


headers = {'user-agent': ua.random}

In [69]:
url_api

'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt=1636041600&endAt=1636991999&limit=30&page=1'

In [75]:
url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt=1636041600&endAt=1636991999&limit=30&page=1'
res = requests.get(url_api, headers=headers)
res_json = res.json()

In [83]:
last_page = res_json['items']['last_page']

In [86]:
import time
import tqdm

In [99]:
news_data_list = []

for i in tqdm.tqdm(range(2,20)):
  url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt=1636041600&endAt=1636991999&limit=30&page=1'
  res = requests.get(url_api, headers=headers)
  res_json = res.json()
  news_data_list.append(res_json)
  time.sleep(1)

100%|██████████| 18/18 [00:19<00:00,  1.07s/it]


In [ ]:
article_content

In [102]:

news_extracted_list=[]

for chunk in new_data_list:
  chunk = new_data_list[0]
  article_content = chunk['items']['data']
  data_dict = [{'id':x['newsId'], 'title':x['title'], 'summary':x['summary'], 'unix_timestamp':x['publishAt']} for x in article_content]
  news_extracted_list.extend(data_dict)


In [105]:
news_extracted_list

[{'id': 4771609,
  'summary': '偉詮電 (2436-TW) 今 (15) 日公布 10 月自結，稅後純益 0.53 億元，年增 179%，每股稅後純益 0.3 元，累計前 10 月稅後純益 6.2 億元，年增 204%，每股稅後純益 3.5 元；受惠 USB PD 出貨續旺，加上 10 月開始漲',
  'title': '偉詮電10月獲利年增179% 前10月EPS 3.5元',
  'unix_timestamp': 1636968032},
 {'id': 4771606,
  'summary': '嘉晶 (3016-TW) 今 (15) 日表示，部分客戶訂單已看至明年底，明年將持續調漲價格，對展望樂觀看待，營收將有雙位數成長、毛利也將續揚，並看好元宇宙將推升伺服器需求，對化合物半導體來說是不錯機會。',
  'title': '〈漢民集團法說〉嘉晶明年營收再雙位數成長 元宇宙是不錯的機會',
  'unix_timestamp': 1636967907},
 {'id': 4771600,
  'summary': '國賓飯店 (2704-TW) 的台北館及高雄館將分別投入危老改建，而欣陸 (3703-TW) 旗下大陸建設與國賓及厚生 (2107-TW) 今 (15) 日正式簽約，將就高雄國賓大飯店現址購入部分土地，並簽約後續的申請危老合建開發案，',
  'title': '高雄國賓吹熄燈號 欣陸攜手厚生投入危老合建案 擬2024年推案',
  'unix_timestamp': 1636967740},
 {'id': 4771569,
  'summary': '航空客運市場復甦 台灣虎航開放冬季班表 4 航線 星宇航推曙光包機',
  'title': '〈航空客運迎復甦〉台灣虎航開放4航線 星宇航推曙光包機',
  'unix_timestamp': 1636967066},
 {'id': 4771544,
  'summary': '',
  'title': '【葉錦龍專欄】指數高檔可留意的二類選股方向',
  'unix_timestamp': 1636966570},
 {'id': 4771559,
  'summary': '玉山金控連續 8 年入選「道瓊永續指數」 再創台灣金融業紀錄',
  'tit